# Przygotowanie danych do pliku logów ze sklepu internetowego



In [1]:
import pandas as pd
import numpy as np

### 1. Data Import
- Odczyt danych z pliku

In [2]:
data = pd.read_excel("logTest.xlsx")

### 2. Weryfikacja zgodności z określonymi typami danych i ograniczeniami
- Weryfikacja 'sessionNo'

In [3]:
if all(isinstance(value, int) and value >= 0 for value in data['sessionNo']):
    print("All values are natural numbers")
else:
    print("Error: not natural numbers")

All values are natural numbers


- Weryfikacja 'startHour'

In [4]:
if all(isinstance(value, int) and 0 <= value <= 23 for value in data['startHour']):
    print("All values are natural numbers between 0 and 23")
else:
    print("Error")

All values are natural numbers between 0 and 23


- Weryfikacja 'startWeekday'

In [5]:
if data['startWeekday'].isnull().any():
  print("Error: missing values")
else:
    data['startWeekday'] = pd.to_numeric(data['startWeekday'], errors='coerce')
    valid_weekdays = set({1, 2, 3, 4, 5, 6, 7})
    if (data['startWeekday'].isin(valid_weekdays)).all():
        print("All values are from the set {1, 2, 3, 4, 5, 6, 7}")
    else:
        print("Error: invalid values")

All values are from the set {1, 2, 3, 4, 5, 6, 7}


- Weryfikacja 'duration'

In [6]:
if all(isinstance(value, float) and value >= 0 for value in data['duration']):
    print("All values are float numbers")
else:
    print("Error: not float numbers")

All values are float numbers


- Weryfikacja 'cCount'

In [7]:
if all(isinstance(value, int) and value >= 0 for value in data['cCount']):
    print("All values are natural numbers")
else:
    print("Error: not natural numbers")

All values are natural numbers


- Weryfikacja 'bCount'

In [8]:
if all(isinstance(value, int) and value >= 0 for value in data['bCount']):
    print("All values are natural numbers")
else:
    print("Error: not natural numbers")

All values are natural numbers


- Weryfikacja 'order'

In [9]:
if all(isinstance(value, str) for value in data['order']):
    print("All values are characters")
else:
    print("Error: not characters")

All values are characters


### 3. Korekta i uzupełnianie brakujących danych
- Uzupełnianie danych 'cMinPrice' 'cMaxPrice' 'cSumPrice' 'bMinPrice' 'bMaxPrice' 'bSumPrice'

W przypadku gdy liczba produktów, które zostały przeglądnięte przez kupującego = 0, wypełniamy dane cenowe zerami

In [10]:
data_filled = data.copy()
def zero_or_current(row, count_column_name, column_name):
    if (row[count_column_name] == 0) or (row[count_column_name] == 1 and row[column_name] == '?'):
        return 0
    else:
        return float(row[column_name])


data_filled['cMinPrice'] = data_filled.apply(lambda row: zero_or_current(row, 'cCount', 'cMinPrice'), axis=1)
data_filled['cMaxPrice'] = data_filled.apply(lambda row: zero_or_current(row, 'cCount', 'cMaxPrice'), axis=1)
data_filled['cSumPrice'] = data_filled.apply(lambda row: zero_or_current(row, 'cCount', 'cSumPrice'), axis=1)

data_filled['bMinPrice'] = data_filled.apply(lambda row: zero_or_current(row, 'bCount', 'bMinPrice'), axis=1)
data_filled['bMaxPrice'] = data_filled.apply(lambda row: zero_or_current(row, 'bCount', 'bMaxPrice'), axis=1)
data_filled['bSumPrice'] = data_filled.apply(lambda row: zero_or_current(row, 'bCount', 'bSumPrice'), axis=1)

data = data_filled

- Korekcja błędnych danych 'cSumPrice' 'bSumPrice'

Suma wszystkich produktów, które klient przeglądał, musi być większa lub równa 
minimalnej cenie pomnożonej przez ilość produktów minus jeden, powiększonej o maksymalną cenę produktu. 
Suma produktów może również być równa tej wartości

In [11]:
def max_or_sum(row, count_column_name, column_sum, column_min, column_max):
    if(row[count_column_name] == 1):
        return float(row[column_max])
    elif ((row[count_column_name] > 1) & (float(row[column_sum]) < float(row[column_min])*(row[count_column_name]-1)+float(row[column_max]))): 
        return float(row[column_min])*(row[count_column_name]-1)+float(row[column_max])
    else:
        return float(row[column_sum])


data_filled['cSumPrice'] = data.apply(lambda row: max_or_sum(row, 'cCount', 'cSumPrice', 'cMinPrice', 'cMaxPrice'), axis=1)

data_filled['bSumPrice'] = data.apply(lambda row: max_or_sum(row, 'bCount', 'bSumPrice', 'bMinPrice', 'bMaxPrice'), axis=1)

data = data_filled

- Uzupełnianie danych 'bStep'

In [12]:
def update_bStep(data):
    data_grouped = data.groupby('sessionNo')
    
    for session_no, group_data in data_grouped:
        
        if group_data['bStep'].eq("?").all():
            for index, row in group_data.iterrows():
                data.at[index, 'bStep'] = 1
                
        else:
            new_val_bStep = '?'
            for index, row in group_data.iterrows():
                if data.at[index, 'bStep'] != '?':
                    new_val_bStep = data.at[index, 'bStep']
                elif data.at[index, 'bStep'] == '?' and new_val_bStep != '?':
                    data.at[index, 'bStep'] = new_val_bStep
                else:
                    while new_val_bStep == '?':
                        for index2, row2 in group_data.iterrows():
                            new_val_bStep = data.at[index2, 'bStep']
                            if new_val_bStep != 0 and new_val_bStep != '?':
                                break
                    data.at[index, 'bStep'] = new_val_bStep
        
    return data


data_filled = update_bStep(data.copy()) 

data_filled['bStep'][0:20]
data = data_filled

- Uzupełnianie danych 'onlineStatus'


Ponieważ dla danej kolumny istnieją dwie opcje, ale wśród znanych danych występuje tylko opcja "yes", można założyć, że brakujące dane odpowiadają za przeciwną kategorię, czyli "no"

In [13]:
def update_onlineStatus(data):
    data['onlineStatus'] = data['onlineStatus'].replace('?', 'n')
    return data

data = update_onlineStatus(data_filled)
data['onlineStatus']

0      n
1      y
2      y
3      y
4      y
      ..
713    y
714    y
715    y
716    y
717    y
Name: onlineStatus, Length: 718, dtype: object

- Uzupełnianie danych 'availability'

In [14]:
def update_availability(data):
    data_grouped = data.groupby('sessionNo')
    
    for session_no, group_data in data_grouped:
        
        if group_data['availability'].eq("?").all():
            for index, row in group_data.iterrows():
                data.at[index, 'availability'] = 'completely not orderable'
                
        else:
            new_val_avail = '?'
            for index, row in group_data.iterrows():
                if data.at[index, 'availability'] != '?':
                    new_val_avail = data.at[index, 'availability']
                elif data.at[index, 'availability'] == '?' and new_val_avail != '?':
                    data.at[index, 'availability'] = new_val_avail
                else:
                    while new_val_avail == '?':
                        for index2, row2 in group_data.iterrows():
                            new_val_avail = data.at[index2, 'availability']
                            if new_val_avail != '?':
                                break
                    data.at[index, 'availability'] = new_val_avail
        
    return data


data_filled = data.copy()
data_filled = update_availability(data.copy()) 

data_filled['availability'][0:20]
data = data_filled

- Uzupełnianie danych 'customerNo'

In [15]:
def update_customerNo(data):

    grouped = data.groupby('sessionNo')
    used_values = set()

    for session_no, group_data in grouped:
        used_values.update(group_data['customerNo'].unique())
        
    used_values = {v for v in used_values if v != "?" and isinstance(v, int)}
    for session_no, group_data in grouped:
        if "?" in group_data['customerNo'].values:
            non_question_values = group_data[group_data['customerNo'] != "?"]['customerNo']
            if not non_question_values.empty:
                replacement_value = non_question_values.iloc[0]
            else:
                replacement_value = None
                for val in range(1, len(data)):
                    if val not in used_values:
                        replacement_value = val
                        break
                
            data.loc[group_data.index[group_data['customerNo'] == "?"], 'customerNo'] = replacement_value
            used_values.add(replacement_value)

    return data

data_filled = data.copy()
data_filled = update_customerNo(data.copy())
data = data_filled

Brakujące dane w kolumnach 'maxVal' 'accountLifetime' są wypełniane na podstawie wartości średniej

- Funkcja uzupełniania danych

In [16]:
from sklearn.impute import SimpleImputer
pd.set_option('future.no_silent_downcasting', True)

def impute_and_convert_to_int(data, column_name):
  data_filled = data.copy()
  data_filled[column_name] = data_filled[column_name].replace('?', np.nan)
    
  imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    
  data_filled[column_name] = imputer.fit_transform(data_filled[[column_name]])
  data_filled[column_name] = data_filled[column_name].astype(int)
    
  return data_filled

- Uzupełnianie danych 'maxVal'

In [17]:
data = impute_and_convert_to_int(data.copy(), 'maxVal')

- Uzupełnianie danych 'customerScore'

In [18]:
data = impute_and_convert_to_int(data.copy(), 'customerScore')

- Uzupełnianie danych 'accountLifetime'

In [19]:
data = impute_and_convert_to_int(data.copy(), 'accountLifetime')

- Uzupełnianie danych 'payments'

In [20]:
data = impute_and_convert_to_int(data.copy(), 'payments')

- Uzupełnianie danych 'age'

In [21]:
data = impute_and_convert_to_int(data.copy(), 'age')

- Uzupełnianie danych 'address'

In [22]:
data = impute_and_convert_to_int(data.copy(), 'address')

- Uzupełnianie danych 'lastOrder'

In [23]:
data = impute_and_convert_to_int(data.copy(), 'lastOrder')

In [24]:
data.to_excel("logTest.xlsx", index=False)  